In [6]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure as Figure
import seaborn as sns
import src.Other.Colours as CP
import numpy as np
import pandas as pd
import math

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# https://pandas.pydata.org/pandas-docs/stable/visualization.html
# http://pandas.pydata.org/pandas-docs/version/0.13/visualization.html
# https://matplotlib.org/users/pyplot_tutorial.html

In [11]:
def palette_test(colour):
    t = [t/5.0 for t in range(30)]
    fig = Figure(figsize=(12.0, 15.0), dpi=100) 
    canvas = FigureCanvas(fig)
    direc = 'src/Other/PaletteOutput/'

    functions = [math.cos, math.sin, math.tan, lambda x: 1.5*x, 
                 math.exp, math.sqrt, math.ceil,
                 math.floor, lambda x: x**2, lambda x: x**3]

    for index, fn in enumerate(functions):
        palette = colour+"_"+str(index)
        ax = fig.add_subplot(5, 2, index+1)
        ax.set_title(palette)
        f = [fn(x) for x in t]
        ax.plot(t, f, color=getattr(CP, palette), markersize=5.0) 

    fig.tight_layout()
    fig.savefig(direc + colour + '_PALETTE.svg')
    
    plt.title(colour + ' PALETTE')
    for index, fn in enumerate(functions):
        palette = colour+"_"+str(index)
        f = [fn(x) for x in t]
        plt.plot(t, f, color=getattr(CP, palette))

    plt.ylim(-15, 20)  # limits the y-axis
    plt.tight_layout()
    plt.savefig(direc + colour + '_OVERLAY.svg')

plt.style.use(plt.style.available[14])  # 5, 14, 22
sns.set_context("paper")
print("Using style '{}'".format(plt.style.available[14]))
palettes = ["KIKI", "PONYO"]
for x in palettes: 
    palette_test(x)

Using style 'seaborn-notebook'


<Figure size 576x396 with 1 Axes>

In [13]:
# Changing this variable changes which file to parse.
# Makes testing/presenting easier, since less code has to be changed.
FILENUM = 1

In [14]:
from random import shuffle, random

from src.EA_Methods.ParentSelectionMethods import tournament as PSM
from src.EA_Methods.RecombinationMethods import recombination_cut_crossover as RM
from src.EA_Methods.MutationMethods import permutation_swap as MM
from src.EA_Methods.SurvivorSelectionMethods import replacement as SSM
from src.Setups.TSP.TSP import read_tsp_file as parse_file
from src.Setups.TSP.TSP import random_initialization as initialize
from src.Setups.TSP.TSP import euclidean_distance as eval_fitness
from src.Setups.TSP import TSP as TSP
from src.EA_Methods import ParentSelectionMethods as ParentSelectionMethods
from src.EA_Methods import SurvivorSelectionMethods as SurvivorSelectionMethods

genome_length       =      parse_file(FILENUM)
generation_limit    =      100

population_size     =      60
mating_pool_size    =      population_size//2 if (population_size//2) % 2 == 0 else (population_size//2)+1  # has to be even
tournament_size     =      population_size//10

mutation_rate       =      0.2
crossover_rate      =      0.9
crossover_point     =      genome_length//3

ParentSelectionMethods.tournament_size = tournament_size



In [17]:
def dict_to_df(d):
    data = pd.DataFrame.from_dict(d, orient='index')
    data.columns = ['x', 'y']
    return data


df = dict_to_df(TSP.LOCATIONS)
df.plot.scatter(x='x', y='y')


<Figure size 576x396 with 1 Axes>

In [6]:
def evolution_algorithm(maximize, print_gens=True, display_freq=None):
    if display_freq is None: display_freq = generation_limit
    
    # Modular function declarations
    op = max if maximize else min
    ParentSelectionMethods.op = op
    SurvivorSelectionMethods.op = op

    # Initialize Population
    population = initialize(population_size, genome_length)
    fitness = [eval_fitness(i) for i in population]
    
    if print_gens:
        
        # TODO - Cool display things go here.
        pass

    # Evolution starts
    for generation in range(generation_limit):

        # Generation Info
        if print_gens:
            print("Generation: {}\n - Best fitness: {}\n - Avg. fitness: {}\n".format(
                generation+1, op(fitness), sum(fitness)/len(fitness))
            )

        # Parent Selection
        parents_index = PSM(fitness, mating_pool_size)
        shuffle(parents_index)

        # Recombination
        offspring = []
        for i in range(0, mating_pool_size, 2):
            if random() < crossover_rate:
                off1, off2 = RM(population[parents_index[i]], population[parents_index[i+1]], crossover_point)
            else:
                off1, off2 = population[parents_index[i]].copy(), population[parents_index[i+1]].copy()
            offspring.append(off1)
            offspring.append(off2)

        # Mutations Selection
        offspring = [MM(i) if random() < mutation_rate else i for i in offspring]
        offspring_fitness = [eval_fitness(i) for i in offspring]

        # Survivor Selection
        population, fitness = SSM(population, fitness, offspring, offspring_fitness)
        
        if print_gens and (generation % display_freq == 0):
            # TODO - Cool display things go here.
            pass
    # Evolution ends
        
    # Final Fitness Info
    op_fit = op(fitness)
    optimal_solutions = [i + 1 for i in range(population_size) if fitness[i] == op_fit]
    print("Best solution fitness:", op_fit, "\nNumber of optimal solutions: ", len(optimal_solutions), '/', population_size)
    print("Best solution indexes:", optimal_solutions)

In [7]:
evolution_algorithm(False, True, min(int(generation_limit * 0.1), 50))

Generation: 1
 - Best fitness: 87488.67715545303
 - Avg. fitness: 110374.79998824821

Generation: 2
 - Best fitness: 85233.7176660953
 - Avg. fitness: 101713.87300574793

Generation: 3
 - Best fitness: 83720.194467435
 - Avg. fitness: 94167.58956247453

Generation: 4
 - Best fitness: 82392.93625588849
 - Avg. fitness: 89423.02066833615

Generation: 5
 - Best fitness: 78338.5964852943
 - Avg. fitness: 86171.44618082196

Generation: 6
 - Best fitness: 78338.5964852943
 - Avg. fitness: 82816.70401675724

Generation: 7
 - Best fitness: 78338.5964852943
 - Avg. fitness: 80466.52750500609

Generation: 8
 - Best fitness: 77502.4158239798
 - Avg. fitness: 79251.34577990859

Generation: 9
 - Best fitness: 75034.1146014247
 - Avg. fitness: 78887.81678217626

Generation: 10
 - Best fitness: 75034.1146014247
 - Avg. fitness: 80216.03829302365

Generation: 11
 - Best fitness: 75034.1146014247
 - Avg. fitness: 81823.12692076924

Generation: 12
 - Best fitness: 75034.1146014247
 - Avg. fitness: 80204


Generation: 75
 - Best fitness: 43131.26018456995
 - Avg. fitness: 44461.36623263546

Generation: 76
 - Best fitness: 43131.26018456995
 - Avg. fitness: 45381.34579909768

Generation: 77
 - Best fitness: 43131.26018456995
 - Avg. fitness: 44211.75608808558

Generation: 78
 - Best fitness: 43131.26018456995
 - Avg. fitness: 44470.358703962986

Generation: 79
 - Best fitness: 43131.26018456995
 - Avg. fitness: 45499.98770360811

Generation: 80
 - Best fitness: 43131.26018456995
 - Avg. fitness: 44188.0187924514

Generation: 81
 - Best fitness: 43131.26018456995
 - Avg. fitness: 45019.44987872067

Generation: 82
 - Best fitness: 41251.5186202691
 - Avg. fitness: 44831.80481586044

Generation: 83
 - Best fitness: 41251.5186202691
 - Avg. fitness: 44544.28240688417

Generation: 84
 - Best fitness: 41251.5186202691
 - Avg. fitness: 43589.25755536207

Generation: 85
 - Best fitness: 41251.5186202691
 - Avg. fitness: 43352.6291045197

Generation: 86
 - Best fitness: 41251.5186202691
 - Avg. f